In [1]:
import numpy as np
import pystan
import arviz
import matplotlib.pyplot as plt

In [2]:
extracted_coefficients_directory = '../../../../data/commaai/extracted_coefficients/20201021_unrestr_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path)
tBB = B_zeta.T.dot(B_zeta)
n = B_zeta.shape[0]
p = B_zeta.shape[1]
X = B_zeta

In [ ]:
hmc_code = '''


functions {
    vector S_xtheta(vector lambda, matrix X, int p, int n) {
        vector[n] s_is;
        for (N in 1:n) {
          s_is[N] = 1/sqrt(1 + sum((row(X, N).*square(to_row_vector(lambda))).*(row(X, N)))); 
        } 
        return(s_is);
    }
    }
data {
  int<lower=1> n; // Number of data
  int<lower=1> p; // Number of covariates
  matrix[n,p] X;  // n-by-p design matrix
  real y[n];      // n-dimensional response vector
}


parameters {
  vector[p] beta;
  vector<lower=0>[p] lambda;
  real<lower=0> tau;
}

transformed parameters {
  vector[n] theta ;
  vector[n] S ;
  vector[n] Var ;
  S = S_xtheta(lambda, X, p, n);
  theta = S .* (X * beta);
  Var = square(S);
}

model {
  tau ~ cauchy(0, 1);
  lambda ~ cauchy(0, tau);
  beta ~ normal(0, square(lambda)); 
  y ~ normal(theta, Var);
}'''

In [ ]:
hmc_dat = {'n': n,
           'p': p,
           'X': X,
           'y': z}

In [ ]:
sm = pystan.StanModel(model_code = hmc_code, verbose = True)
print('finished compiling')
print('start sampling')
fit = sm.sampling(data = hmc_dat,
                  iter = 2000, 
                  chains = 1, 
                  verbose = True)
print('finished sampling')

In [ ]:
#la = fit.extract(permuted=True) 
la = fit.extract(permuted=True)

In [ ]:
plt.plot(la['lambda'])

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('horseshoe_hmc_unf_1000.p', 'wb') as fp:
    pickle.dump(la, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
la['lambda'].shape

In [ ]:
for i in range(0, la['lambda'].shape[1] - 1):
    plt.plot(la['lambda'][:,i])

In [ ]:
la['lambda'].shape

In [ ]:
fit.plot()

In [ ]:
arviz.plot_trace(fit)

In [ ]:
betas = np.mean(la['beta'], axis = 0)
betas_sd = np.std(la['beta'], axis = 0)

In [ ]:
tau = np.mean(la['tau'], axis = 0)
tau_sd = np.std(np.log(la['tau']), axis = 0)

In [ ]:
lambdas = np.mean(la['lambda'], axis = 0)
lambdas_sd = np.std(np.log(la['lambda']**2), axis = 0)

In [ ]:
mu_ts = np.load('../../../../data/commaai/va/filtered_gaussian_resampled/Horseshoe/mu_ts.npy')

In [ ]:
B_ts = np.load('../../../../data/commaai/va/filtered_gaussian_resampled/Horseshoe/B_ts.npy')
d_ts = np.load('../../../../data/commaai/va/filtered_gaussian_resampled/Horseshoe/d_ts.npy')

In [ ]:
B_ts = np.mean(B_ts.reshape(29001, 21, 19)[20000:,:,:], axis = 0)
d_ts = np.mean(d_ts.reshape(29001, 21)[20000:,:], axis = 0)

In [ ]:
va_sd = np.sqrt(np.diag((B_ts.dot(B_ts.T) + d_ts**2)))

In [ ]:
mu_ts_m = np.mean(mu_ts.reshape(29001, 21)[25000:,:], axis = 0)
mu_ts_sd = np.std(mu_ts.reshape(29001, 21)[25000:,:], axis = 0)

In [ ]:
plt.scatter(mu_ts_m[0:10], betas)
ident = [-.5, .5]
plt.plot(ident,ident)

In [ ]:
plt.scatter(va_sd[10:20], lambdas_sd)
ident = [0, .5]
plt.plot(ident,ident)

In [ ]:
plt.scatter(va_sd[0:10], betas_sd)
ident = [0, .5]
plt.plot(ident,ident)

In [ ]:
plt.scatter(np.exp(0.5*mu_ts_m[10:20]), lambdas)
ident = [-.5, .5]
plt.plot(ident,ident)

In [ ]:
plt.scatter(np.exp(mu_ts_m[20]), tau)
ident = [-.5, .5]
plt.plot(ident,ident)

In [ ]:
plt.scatter(va_sd[20], tau_sd)
ident = [0, .5]
plt.plot(ident,ident)